In [11]:
# -*- coding: utf-8 -*-                                                                        
import numpy as np
import itertools as it
import onnx
from onnx import helper, checker
from onnx import TensorProto
import os, psutil

# import autodice functions .
from code_generator import CppFile
from cpp_generator import *
from onnx_split import *
from data_json import *
from interface import *

In [5]:
origin_model = "bvlcalexnet-9.onnx"
input_model = format_onnx(origin_model)  # name all layers according to their output tensor name
model =  onnx.load(input_model) 
model_len = len(model.graph.node)

resourceid = { 1:'nx01_arm012345', 2:'nx01_gpu'}
platforms = ['nx01']
cnn_map = {
        'nx01_arm012345': ['conv1_1', 'conv1_2', 'norm1_1', 
            'pool1_1', 'conv2_1', 'conv2_2', 'norm2_1', 'pool2_1', 'conv3_1', 
            'conv3_2', 'conv4_1', 'conv4_2', 'conv5_1', 'conv5_2', 'pool5_1', 
            'OC2_DUMMY_0', 'fc6_1', 'fc6_2'], 
        'nx01_gpu': ['fc6_3', 'fc7_1', 'fc7_2', 'fc7_3', 'fc8_1', 'prob_1']}

# CNN Mapping:  
# {'nx01_arm012345': ['conv1_1', 'conv1_2', 'norm1_1', 
# 'pool1_1', 'conv2_1', 'conv2_2', 'norm2_1', 'pool2_1', 'conv3_1', 
# 'conv3_2', 'conv4_1', 'conv4_2', 'conv5_1', 'conv5_2', 'pool5_1', 
# 'OC2_DUMMY_0', 'fc6_1', 'fc6_2'], 
# 'nx01_gpu': ['fc6_3', 'fc7_1', 'fc7_2', 'fc7_3', 'fc8_1', 'prob_1']}




 MODEL has total 24 layers.


In [14]:
# call interface function to generate onnx models
# Check whether the specified path exists or not
output_dirs= './models'
if not os.path.exists(output_dirs):
  # Create a new directory because it does not exist 
  os.makedirs(output_dirs)
  print("The output directory %s is created!" % (output_dirs))

start_time = time.time()
InputSpecs = Interface(model=input_model, mappings=cnn_map, platforms=platforms)
print ("Front End time: %f (s)"%(time.time() - start_time))
output_dirs = 'models'



The output directory ./models is created!
Total GPU: 1, Total CPU: 6
Consistency Check Pass.
Devices: 1
Generate  2  Sub-Models.
Front End time: 5.287814 (s)


In [15]:
# Generate cpp code 
GenerateCode = EngineCode(
    CppName = "./models/multinode",
    Platforms = InputSpecs.platforms,
    NodesList = InputSpecs.nodes,
    ComputingNodes = InputSpecs.computingnodes,
    ValueInfos = InputSpecs.value_map,
    Inputs = InputSpecs.inputs,
    Outputs = InputSpecs.outputs,
    Benchmark = False)
